In [15]:
import os
import time
import pickle
import numpy as np
import tensorflow as tf
from keras.layers import Dense
import matplotlib.pyplot as plt
from keras.layers import MaxPool2D
from keras.models import Sequential
from keras.layers import Convolution2D
from tensorflow.keras import datasets, layers, models
from keras.preprocessing.image import ImageDataGenerator

*URL of Data :*
https://drive.google.com/drive/folders/17B4x0i-PlIa5KDsXO-8d8c9eY-VVMFau?usp=share_link

In [6]:
TrainingImagePath='/content/drive/MyDrive/Smart attendance dataset'
train_datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)
test_datagen = ImageDataGenerator()
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')
test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')
test_set.class_indices

Found 995 images belonging to 10 classes.
Found 995 images belonging to 10 classes.


{'Abir': 0,
 'Fahim': 1,
 'Hemel': 2,
 'Nipa': 3,
 'Rupak': 4,
 'Sabiqul': 5,
 'Shepon': 6,
 'Tama': 7,
 'Tamim': 8,
 'Tarup': 9}

In [7]:
# class_indices have the numeric tag for each face
TrainClasses=training_set.class_indices
ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName
with open("ResultsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)
print("Mapping of Face and its ID",ResultMap)
OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

Mapping of Face and its ID {0: 'Abir', 1: 'Fahim', 2: 'Hemel', 3: 'Nipa', 4: 'Rupak', 5: 'Sabiqul', 6: 'Shepon', 7: 'Tama', 8: 'Tamim', 9: 'Tarup'}

 The Number of output neurons:  10


In [8]:
#STEP_1 Initializing the Convolutional Neural Network
classifier= Sequential()

# STEP_2 MAX Pooling
classifier.add(MaxPool2D(pool_size=(2,2)))

# ADDITIONAL LAYER of CONVOLUTION for better accuracy
classifier.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))

classifier.add(MaxPool2D(pool_size=(2,2)))

# STEP_3 FLattening
classifier.add(Flatten())

# STEP_4 Fully Connected Neural Network
classifier.add(Dense(64, activation='relu'))

classifier.add(Dense(OutputNeurons, activation='softmax'))

# Compiling the CNN
classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])


In [10]:
# Measuring the time taken by the model to train
StartTime=time.time()

# Starting the model training
classifier.fit_generator(
                    training_set,
                    steps_per_epoch=30,
                    epochs=10,
                    validation_data=test_set,
                    validation_steps=10)

EndTime=time.time()
print(" Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ######')


<ipython-input-10-545687561585>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/10
30/30 [==============================] - 50s 2s/step - loss: 0.3627 - accuracy: 0.8894 - val_loss: 0.0294 - val_accuracy: 0.9969
Epoch 2/10
30/30 [==============================] - 9s 315ms/step - loss: 0.0541 - accuracy: 0.9871 - val_loss: 0.0374 - val_accuracy: 0.9875
Epoch 3/10
30/30 [==============================] - 10s 346ms/step - loss: 0.0076 - accuracy: 0.9989 - val_loss: 0.0085 - val_accuracy: 0.9969
Epoch 4/10
30/30 [==============================] - 9s 301ms/step - loss: 0.0126 - accuracy: 0.9957 - val_loss: 0.0116 - val_accuracy: 0.9969
Epoch 5/10
30/30 [==============================] - 9s 305ms/step - loss: 0.0349 - accuracy: 0.9882 - val_loss: 0.2728 - val_accuracy: 0.9156
Epoch 6/10
30/30 [==============================] - 10s 342ms/step - loss: 0.1733 - accuracy: 0.9538 - val_loss: 0.0096 - val_accuracy: 1.0000
Epoch 7/10
30/30 [==============================] - 9s 310ms/step - loss: 0.0101 - accuracy: 0.9979 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 

 *Now We Will Test The Model*

 *URL of Test Data:* /content/drive/MyDrive/test

In [17]:
def predict(ImagePath,classifier):
  ResultMap={0:'Abir', 1:'Fahim', 2:'Hemel', 3:'Nipa', 4:'Rupak', 5:'Sabiqul', 6:'Shepon', 7:'Tama', 8:'Tamim', 9:'Tarup'}
  photo_names= os.listdir(ImagePath)
  for image_name in photo_names:
    test_image=image.load_img(ImagePath+'/'+image_name,target_size=(64, 64))
    test_image=image.img_to_array(test_image)
    test_image=np.expand_dims(test_image,axis=0)
    result=classifier.predict(test_image,verbose=0)
    print('***'*10)
    print('Prediction is: ',ResultMap[np.argmax(result)])
    if ResultMap[np.argmax(result)] in image_name:
      print('correct :)')
    else:
      print('not correct  :(')

predict('/content/drive/MyDrive/test',classifier)

******************************
Prediction is:  Hemel
correct :)
******************************
Prediction is:  Abir
correct :)
******************************
Prediction is:  Fahim
correct :)
******************************
Prediction is:  Rupak
correct :)
******************************
Prediction is:  Sabiqul
correct :)
******************************
Prediction is:  Shepon
correct :)
******************************
Prediction is:  Nipa
correct :)
******************************
Prediction is:  Tamim
correct :)
******************************
Prediction is:  Tama
correct :)
******************************
Prediction is:  Tarup
correct :)
